# Build a Dataset for Modeling

In [20]:
quarter = "2012Q3"
directory = "Data/"

In [28]:
import pandas as pd
pd.set_option("display.max_row",500)
pd.set_option("display.max_columns",500)

import warnings
warnings.filterwarnings("ignore")

import numpy as np

## Load Dataset

In [21]:
filename=directory+"Performance_"+quarter+".txt"
df_perf = pd.read_csv(filename, sep="|",header=None)
df_perf.columns = ["LOAN_ID", "Monthly.Rpt.Prd", "Servicer.Name",
    "LAST_RT", "LAST_UPB", "Loan.Age", "Months.To.Legal.Mat",
    "Adj.Month.To.Mat", "Maturity.Date", "MSA",
    "Delq.Status", "MOD_FLAG", "Zero.Bal.Code",
    "ZB_DTE", "LPI_DTE", "FCC_DTE","DISP_DT",
    "FCC_COST", "PP_COST", "AR_COST", "IE_COST",
    "TAX_COST", "NS_PROCS",
    "CE_PROCS", "RMW_PROCS", "O_PROCS", "NON_INT_UPB",
    "PRIN_FORG_UPB_FHFA", "REPCH_FLAG",
    "PRIN_FORG_UPB_OTH", "TRANSFER_FLG"]

In [22]:
filename=directory+"Acquisition_"+quarter+".txt"
df_acq = pd.read_csv(filename, sep="|",header=None)
df_acq.columns = ["LOAN_ID", "ORIG_CHN", "Seller.Name", "ORIG_RT",
    "ORIG_AMT", "ORIG_TRM", "ORIG_DTE","FRST_DTE", "OLTV", "OCLTV", "NUM_BO", "DTI",
    "CSCORE_B", "FTHB_FLG", "PURPOSE", "PROP_TYP", "NUM_UNIT", "OCC_STAT", "STATE", "ZIP_3",
    "MI_PCT", "Product.Type", "CSCORE_C", "MI_TYPE", "RELOCATION_FLG"]

#### Cleanup Needed - Found through EDA and Modeling

In [23]:
df_acq.loc[df_acq['FTHB_FLG']=="U","FTHB_FLG"]=None
df_acq.loc[df_acq['OCC_STAT']=="U","OCC_STAT"]=None

## Drop Records with Missing Data

In [24]:
# drop records with missing data
vars_to_use = ["ORIG_CHN", "Seller.Name", "ORIG_RT", "ORIG_AMT", "ORIG_TRM",
                   "OLTV", "OCLTV", "DTI", "NUM_BO", "CSCORE_B", 
                   "FTHB_FLG", "PURPOSE", "PROP_TYP", "NUM_UNIT", "OCC_STAT", 
                   "STATE", "RELOCATION_FLG"]
df_acq = df_acq.dropna(subset=vars_to_use)

## Build Dataset:  Sample, Add Max Delinquency, Merge, and Clean Dataset

In [25]:
# get max delinquency value for each loan in sample
df = pd.merge(df_acq, df_perf[df_perf["Delq.Status"]!="X"], on=["LOAN_ID"])
df["Delq.Status"] = df["Delq.Status"].apply(pd.to_numeric)
delq_by_loan = df.groupby(["LOAN_ID"])["Delq.Status"].max()
delq_by_loan.rename("Max.Delq",inplace=True)
delq_by_loan = delq_by_loan.to_frame()

## Create Target Variable

In [26]:
# create target variable to indicate default (3 months or more of non-payment)
df = pd.merge(df_acq, delq_by_loan, on=["LOAN_ID"])
df["target"] = np.where(df["Max.Delq"] > 2, "yes", "no")

## Write File

In [27]:
# create file
df.to_csv(directory+"Adjusted_Acquisition_"+quarter+".txt",index=False)